# AST-P | 5-Fold Cross-Validation Pipeline for TB Screening (Holdout Strategy)
## Binary Classification: PTB (Label 1) vs Non-PTB (Label 0)

Pipeline ประกอบด้วย 4 ขั้นตอนหลัก:
1. **Data Preparation** — แบ่ง 20% ออกเป็น Holdout Test ก่อน จากนั้นนำ 80% ที่เหลือทำ Patient-Level 5-Fold CV (สัดส่วน Train 85% / Val 15% ต่อ Fold)
2. **Training Loop** — เทรนโมเดล AST-P ครบ 5 Folds บน train/val ของแต่ละ Fold
3. **Inference & Evaluation** — ทดสอบ Best Model ของแต่ละ Fold บน Holdout Test Set เดียวกัน พร้อม AUROC per Fold và Mean AUROC
4. **Aggregate Results** — สรุปผล Mean ± Std ของ AUROC, Sensitivity, Specificity พร้อม Confusion Matrix

In [1]:
# =============================================================================
# STEP 1: DATA PREPARATION — Holdout 20% Test + Patient-Level 5-Fold CV (80%)
# =============================================================================
# กลยุทธ์:
#   1. แบ่ง Patient ทั้งหมดเป็น Holdout Test 20% + Pool 80% (Stratified)
#   2. ใช้ StratifiedKFold(5) บน Pool 80%  → แต่ละ Fold: Train ~85% / Val ~15%
#   3. บันทึก JSON มาตรฐาน AST สำหรับ: holdout_test / fold_X_train / fold_X_val
# =============================================================================

import os
import json
import glob
import numpy as np
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold, train_test_split

# --------------------------------------------------------------------------- #
# 1.1  กำหนด Paths
# --------------------------------------------------------------------------- #
BASE_DIR   = r"z:\AST-With-TB-Classify"
DATA_DIR   = os.path.join(BASE_DIR, "Data")
PTB_DIR    = os.path.join(DATA_DIR, "Cough_PTB")        # Label = 1
NONPTB_DIR = os.path.join(DATA_DIR, "Cough_Non-PTB")    # Label = 0
FOLD_ROOT  = os.path.join(BASE_DIR, "json_folds")        # โฟลเดอร์รองรับ JSON ทุก Fold
N_FOLDS    = 5
RANDOM_STATE = 42

# --------------------------------------------------------------------------- #
# 1.2  รวบรวม Patient ID และไฟล์เสียง
# --------------------------------------------------------------------------- #
def collect_patients(directory, label_str):
    """คืนค่า dict: { patient_id: [{"wav": path, "labels": label_str}, ...] }"""
    patients = defaultdict(list)
    for wav_path in sorted(glob.glob(os.path.join(directory, "*.wav"))):
        filename   = os.path.basename(wav_path)
        patient_id = filename.split("_")[0]
        patients[patient_id].append({
            "wav":    wav_path.replace("\\", "/"),
            "labels": label_str
        })
    return patients

ptb_patients    = collect_patients(PTB_DIR,    label_str="1")
nonptb_patients = collect_patients(NONPTB_DIR, label_str="0")

ptb_ids    = sorted(ptb_patients.keys())
nonptb_ids = sorted(nonptb_patients.keys())

print(f"PTB     patients ({len(ptb_ids):2d}): {ptb_ids}")
print(f"Non-PTB patients ({len(nonptb_ids):2d}): {nonptb_ids}")
print(f"Total patients  : {len(ptb_ids) + len(nonptb_ids)}\n")

# --------------------------------------------------------------------------- #
# 1.3  ขั้นที่ 1 — แบ่ง Holdout Test 20% (Patient-Level Stratified)
# --------------------------------------------------------------------------- #
all_patient_ids  = ptb_ids + nonptb_ids
all_patient_lbls = [1] * len(ptb_ids) + [0] * len(nonptb_ids)

pool_ids, holdout_ids, pool_lbls, holdout_lbls = train_test_split(
    all_patient_ids,
    all_patient_lbls,
    test_size    = 0.20,
    stratify     = all_patient_lbls,
    random_state = RANDOM_STATE
)

pool_ids    = sorted(pool_ids)
holdout_ids = sorted(holdout_ids)

print("=" * 60)
print("STEP 1A — Holdout Test Split (20% of all Patients)")
print("=" * 60)
n_holdout_ptb    = sum(1 for p in holdout_ids if p in ptb_patients)
n_holdout_nonptb = sum(1 for p in holdout_ids if p in nonptb_patients)
n_pool_ptb       = sum(1 for p in pool_ids    if p in ptb_patients)
n_pool_nonptb    = sum(1 for p in pool_ids    if p in nonptb_patients)

print(f"  Holdout Test  : {len(holdout_ids)} patients "
      f"({sum(1 for p in holdout_ids if p in ptb_patients)} PTB + "
      f"{sum(1 for p in holdout_ids if p in nonptb_patients)} Non-PTB)")
print(f"  Pool (80%)    : {len(pool_ids)} patients "
      f"({n_pool_ptb} PTB + {n_pool_nonptb} Non-PTB)")
print(f"  Holdout IDs   : {holdout_ids}")
print(f"  Pool IDs      : {pool_ids}")

# --------------------------------------------------------------------------- #
# 1.4  ขั้นที่ 2 — 5-Fold StratifiedKFold บน Pool 80%
#                   แต่ละ Fold: Train ~85% / Val ~15% ของ Pool
# --------------------------------------------------------------------------- #
pool_ids_arr    = np.array(pool_ids)
pool_lbls_arr   = np.array([1 if p in ptb_patients else 0 for p in pool_ids])

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

fold_assignments = []
for fold_idx, (train_idx, val_idx) in enumerate(skf.split(pool_ids_arr, pool_lbls_arr)):
    train_ids = pool_ids_arr[train_idx].tolist()
    val_ids   = pool_ids_arr[val_idx].tolist()
    fold_assignments.append({
        "fold":  fold_idx + 1,
        "train": sorted(train_ids),
        "val":   sorted(val_ids),
    })

# --------------------------------------------------------------------------- #
# 1.5  Helper Functions
# --------------------------------------------------------------------------- #
def patients_to_records(patient_ids, ptb_dict, nonptb_dict):
    records = []
    for pid in patient_ids:
        if pid in ptb_dict:
            records.extend(ptb_dict[pid])
        elif pid in nonptb_dict:
            records.extend(nonptb_dict[pid])
    return records

def save_json(records, filepath):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w") as f:
        json.dump({"data": records}, f, indent=4)

# --------------------------------------------------------------------------- #
# 1.6  บันทึก Holdout Test JSON
# --------------------------------------------------------------------------- #
holdout_records = patients_to_records(holdout_ids, ptb_patients, nonptb_patients)
holdout_json_path = os.path.join(FOLD_ROOT, "holdout_test.json")
save_json(holdout_records, holdout_json_path)
n_holdout_ptb_files    = sum(1 for r in holdout_records if r["labels"] == "1")
n_holdout_nonptb_files = len(holdout_records) - n_holdout_ptb_files
print(f"\n  Holdout Test JSON → {holdout_json_path}")
print(f"  Files: {len(holdout_records)} total  [{n_holdout_ptb_files} PTB / {n_holdout_nonptb_files} Non-PTB]")

# --------------------------------------------------------------------------- #
# 1.7  บันทึก JSON ของแต่ละ Fold + แสดงสรุป
# --------------------------------------------------------------------------- #
print(f"\n{'=' * 60}")
print("STEP 1B — 5-Fold CV Split (on 80% Pool)")
print(f"({'Approx. Train 85% / Val 15% per Fold based on Pool size'})")
print(f"{'=' * 60}")

for fa in fold_assignments:
    fold_num  = fa["fold"]
    fold_dir  = os.path.join(FOLD_ROOT, f"fold_{fold_num}")

    train_records = patients_to_records(fa["train"], ptb_patients, nonptb_patients)
    val_records   = patients_to_records(fa["val"],   ptb_patients, nonptb_patients)

    save_json(train_records, os.path.join(fold_dir, "train_data.json"))
    save_json(val_records,   os.path.join(fold_dir, "val_data.json"))

    n_train_ptb = sum(1 for r in train_records if r["labels"] == "1")
    n_val_ptb   = sum(1 for r in val_records   if r["labels"] == "1")
    pct_train   = len(fa["train"]) / len(pool_ids) * 100
    pct_val     = len(fa["val"])   / len(pool_ids) * 100

    print(f"\n  --- FOLD {fold_num} ---")
    print(f"  TRAIN : {len(fa['train']):2d} patients ({pct_train:.1f}%) "
          f"[{sum(1 for p in fa['train'] if p in ptb_patients)} PTB + "
          f"{sum(1 for p in fa['train'] if p in nonptb_patients)} Non-PTB] "
          f"→ {len(train_records):3d} files  [{n_train_ptb} PTB / {len(train_records)-n_train_ptb} Non-PTB]")
    print(f"  VAL   : {len(fa['val']):2d} patients ({pct_val:.1f}%)  "
          f"[{sum(1 for p in fa['val'] if p in ptb_patients)} PTB + "
          f"{sum(1 for p in fa['val'] if p in nonptb_patients)} Non-PTB] "
          f"→ {len(val_records):3d} files  [{n_val_ptb} PTB / {len(val_records)-n_val_ptb} Non-PTB]")
    print(f"  IDs   → TRAIN: {fa['train']} | VAL: {fa['val']}")

print(f"\n  TEST  : Shared Holdout — {len(holdout_ids)} patients "
      f"→ {len(holdout_records)} files  "
      f"[{n_holdout_ptb_files} PTB / {n_holdout_nonptb_files} Non-PTB]")
print(f"\n[✓] JSON files saved to: {FOLD_ROOT}")
print("[✓] Data preparation complete.")


PTB     patients ( 7): ['006', '008', '009', '011', '012', '015', '016']
Non-PTB patients ( 8): ['001', '002', '003', '004', '005', '007', '013', '014']
Total patients  : 15

STEP 1A — Holdout Test Split (20% of all Patients)
  Holdout Test  : 3 patients (1 PTB + 2 Non-PTB)
  Pool (80%)    : 12 patients (6 PTB + 6 Non-PTB)
  Holdout IDs   : ['004', '013', '016']
  Pool IDs      : ['001', '002', '003', '005', '006', '007', '008', '009', '011', '012', '014', '015']

  Holdout Test JSON → z:\AST-With-TB-Classify\json_folds\holdout_test.json
  Files: 112 total  [31 PTB / 81 Non-PTB]

STEP 1B — 5-Fold CV Split (on 80% Pool)
(Approx. Train 85% / Val 15% per Fold based on Pool size)

  --- FOLD 1 ---
  TRAIN :  9 patients (75.0%) [5 PTB + 4 Non-PTB] → 209 files  [114 PTB / 95 Non-PTB]
  VAL   :  3 patients (25.0%)  [1 PTB + 2 Non-PTB] → 191 files  [42 PTB / 149 Non-PTB]
  IDs   → TRAIN: ['003', '005', '006', '007', '009', '011', '012', '014', '015'] | VAL: ['001', '002', '008']

  --- FOLD 2 

In [2]:

# =============================================================================
# STEP 2: MODEL CONFIGURATION & 5-FOLD TRAINING LOOP
# =============================================================================
# แต่ละ Fold จะใช้ fold_X/train_data.json และ fold_X/val_data.json
# (จาก STEP 1)  ไม่มี per-fold test — ทุก Fold ทดสอบบน Holdout Test เดียวกัน
#
# Audio Settings (target_length=100 → ~1 วินาที):
#   TRAIN_AUDIO_CONF : FreqMask(48) + TimeMask(20) + Mixup(0.5)
#   EVAL_AUDIO_CONF  : ปิด Augmentation ทั้งหมด
#   Normalization    : AudioSet global mean=-4.2677393, std=4.5689974
#
# AST-P Model:
#   label_dim=2, input_tdim=100, base384, ImageNet + AudioSet pretrained
#
# Training Hyperparameters:
#   epochs=30, lr=1e-5, batch=8, WA: epoch 10→30
#   LR Scheduler: MultiStep (start=5, step=1, decay=0.85)
# =============================================================================

import sys
import os
import pickle
import argparse
import importlib
import torch
import numpy as np
from torch.utils.data import DataLoader

# --------------------------------------------------------------------------- #
# 2.1  เพิ่ม src/ directory เข้า sys.path
# --------------------------------------------------------------------------- #
SRC_DIR = os.path.join(BASE_DIR, "src")
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

import dataloader as ast_dataloader
import models
import traintest as _traintest_module

# Force-reload modules to pick up any file changes
importlib.reload(_traintest_module)
importlib.reload(ast_dataloader)
importlib.reload(models)

from traintest import train

# --------------------------------------------------------------------------- #
# 2.2  Paths
# --------------------------------------------------------------------------- #
LABEL_CSV = os.path.join(BASE_DIR, "class_labels_indices.csv")
EXP_ROOT  = os.path.join(BASE_DIR, "exp", "kfold")
NORM_MEAN = -4.527155
NORM_STD  =  5.118366
# --------------------------------------------------------------------------- #
# 2.3  Audio Configurations
# --------------------------------------------------------------------------- #
TRAIN_AUDIO_CONF = {
    "num_mel_bins": 128,
    "target_length": 100,
    "freqm":        48,
    "timem":        20,
    "mixup":        0.5,
    "dataset":      "audioset",
    "mode":         "train",
    "mean":        NORM_MEAN,
    "std":          NORM_STD,
    "noise":        False,
    'skip_norm': False
}

EVAL_AUDIO_CONF = {
    "num_mel_bins": 128,
    "target_length": 100,
    "freqm":        0,
    "timem":        0,
    "mixup":        0.0,
    "dataset":      "audioset",
    "mode":         "evaluation",
    "mean":        NORM_MEAN,
    "std":          NORM_STD,
    "noise":        False,
    'skip_norm': False
}

# --------------------------------------------------------------------------- #
# 2.4  Training Arguments
# --------------------------------------------------------------------------- #
def make_training_args(fold_exp_dir, n_epochs=20):
    return argparse.Namespace(
        exp_dir           = fold_exp_dir,
        dataset           = "audioset",
        n_class           = 2,
        lr                = 1e-5,
        n_epochs          = n_epochs,
        batch_size        = 8,
        n_print_steps     = 10,
        save_model        = True,
        metrics           = "mAP",
        loss              = "CE",
        warmup            = False,
        lrscheduler_start = 5,
        lrscheduler_step  = 1,
        lrscheduler_decay = 0.85,
        wa                = True,
        wa_start          = 10,
        wa_end            = n_epochs,
    )

# --------------------------------------------------------------------------- #
# 2.5  5-FOLD TRAINING LOOP
# --------------------------------------------------------------------------- #
print("=" * 65)
print("  AST-P  |  5-FOLD CROSS-VALIDATION TRAINING")
print("  Strategy: Shared Holdout Test (20%) + 5-Fold CV on Pool (80%)")
print("=" * 65)
print(f"  Device  : {'CUDA (' + torch.cuda.get_device_name(0) + ')' if torch.cuda.is_available() else 'CPU'}")
print(f"  Epochs  : 20  |  LR: 1e-5  |  Batch: 8  |  WA: epoch 10→20")
print("=" * 65)

for fold in range(1, N_FOLDS + 1):

    print(f"\n{'━'*65}")
    print(f"  ▶  FOLD {fold} / {N_FOLDS}  TRAINING START")
    print(f"{'━'*65}")

    # Paths
    fold_data_dir = os.path.join(FOLD_ROOT, f"fold_{fold}")
    fold_exp_dir  = os.path.join(EXP_ROOT,  f"fold_{fold}")
    os.makedirs(os.path.join(fold_exp_dir, "models"),      exist_ok=True)
    os.makedirs(os.path.join(fold_exp_dir, "predictions"), exist_ok=True)

    train_json = os.path.join(fold_data_dir, "train_data.json")
    val_json   = os.path.join(fold_data_dir, "val_data.json")

    # DataLoaders
    train_dataset = ast_dataloader.AudiosetDataset(
        train_json, audio_conf=TRAIN_AUDIO_CONF, label_csv=LABEL_CSV)
    train_loader  = DataLoader(
        train_dataset, batch_size=8, shuffle=True,
        num_workers=0, pin_memory=torch.cuda.is_available())

    val_dataset = ast_dataloader.AudiosetDataset(
        val_json, audio_conf=EVAL_AUDIO_CONF, label_csv=LABEL_CSV)
    val_loader  = DataLoader(
        val_dataset, batch_size=16, shuffle=False,
        num_workers=0, pin_memory=torch.cuda.is_available())

    print(f"\n  [Data] Train: {len(train_dataset)} files | Val: {len(val_dataset)} files")

    # สร้างโมเดล AST-P ใหม่ทุก Fold (เริ่มจาก pretrained weights)
    audio_model = models.ASTModel(
        label_dim         = 2,
        fstride           = 10,
        tstride           = 10,
        input_fdim        = 128,
        input_tdim        = 100,
        imagenet_pretrain = True,
        audioset_pretrain = True,
        model_size        = "base384"
    )

    args = make_training_args(fold_exp_dir, n_epochs=20)
    with open(os.path.join(fold_exp_dir, "args.pkl"), "wb") as f:
        pickle.dump(args, f)

    print(f"  [Model] AST-P created → output: {fold_exp_dir}")
    print(f"  [Training] Starting {args.n_epochs} epochs...\n")

    train(audio_model, train_loader, val_loader, args)

    print(f"\n  [✓] FOLD {fold} training complete.")
    print(f"      Results  → {fold_exp_dir}/result.csv")
    print(f"      WA Model → {fold_exp_dir}/models/audio_model_wa.pth")

print(f"\n{'═'*65}")
print("  [✓] ALL 5 FOLDS TRAINING COMPLETE")
print(f"{'═'*65}")


z:\AST-With-TB-Classify\venvast\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
z:\AST-With-TB-Classify\venvast\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
z:\AST-With-TB-Classify\venvast\lib\site-packages\torch\cuda\amp\autocast_mode.py:118: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


  AST-P  |  5-FOLD CROSS-VALIDATION TRAINING
  Strategy: Shared Holdout Test (20%) + 5-Fold CV on Pool (80%)
  Device  : CPU
  Epochs  : 20  |  LR: 1e-5  |  Batch: 8  |  WA: epoch 10→20

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  ▶  FOLD 1 / 5  TRAINING START
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
---------------the train dataloader---------------
now using following mask: 48 freq, 20 time
now using mix-up with rate 0.500000
now process audioset
use dataset mean -4.527 and std 5.118 to normalize the input.
number of classes is 2
---------------the evaluation dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process audioset
use dataset mean -4.527 and std 5.118 to normalize the input.
number of classes is 2

  [Data] Train: 209 files | Val: 191 files
---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
frequncey stride=10, time s

z:\AST-With-TB-Classify\venvast\lib\site-packages\torch\cuda\amp\grad_scaler.py:116: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


Epoch: [1][10/27]	Per Sample Total Time 0.93556	Per Sample Data Time 0.00713	Per Sample DNN Time 0.92843	Train Loss 0.6746	
Epoch: [1][20/27]	Per Sample Total Time 0.93139	Per Sample Data Time 0.00648	Per Sample DNN Time 0.92491	Train Loss 0.5570	
start validation
mAP: 0.501353
AUC: 0.422499
Avg Precision: 0.504183
Avg Recall: 1.000000
d_prime: -0.276484
train_loss: 0.521477
valid_loss: 0.704336
validation finished
Epoch-1 lr: 1e-05
epoch 1 training time: 252.144
---------------
2026-02-23 02:42:39.881506
current #epochs=2, #steps=27


z:\AST-With-TB-Classify\venvast\lib\site-packages\torch\cuda\amp\autocast_mode.py:118: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


...

Epoch: [18][4/48]	Per Sample Total Time 1.05372	Per Sample Data Time 0.00449	Per Sample DNN Time 1.04922	Train Loss 0.0872	
Epoch: [18][14/48]	Per Sample Total Time 1.04522	Per Sample Data Time 0.00449	Per Sample DNN Time 1.04073	Train Loss 0.0731	
Epoch: [18][24/48]	Per Sample Total Time 1.04465	Per Sample Data Time 0.00443	Per Sample DNN Time 1.04022	Train Loss 0.0686	
Epoch: [18][34/48]	Per Sample Total Time 1.04620	Per Sample Data Time 0.00446	Per Sample DNN Time 1.04174	Train Loss 0.0765	
Epoch: [18][44/48]	Per Sample Total Time 1.04586	Per Sample Data Time 0.00441	Per Sample DNN Time 1.04145	Train Loss 0.0877	
start validation
mAP: 0.753693
AUC: 0.612500
Avg Precision: 0.507177
Avg Recall: 1.000000
d_prime: 0.404240
train_loss: 0.084328
valid_loss: 0.956626
validation finished
Epoch-18 lr: 1.0276966953088434e-06
epoch 18 training time: 403.754
---------------
2026-02-23 12:45:29.619931
current #epochs=19, #steps=864


z:\AST-With-TB-Classify\venvast\lib\site-packages\torch\cuda\amp\autocast_mode.py:118: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


Epoch: [19][6/48]	Per Sample Total Time 1.04610	Per Sample Data Time 0.00427	Per Sample DNN Time 1.04182	Train Loss 0.0946	
Epoch: [19][16/48]	Per Sample Total Time 1.04185	Per Sample Data Time 0.00424	Per Sample DNN Time 1.03761	Train Loss 0.0928	
Epoch: [19][26/48]	Per Sample Total Time 1.04205	Per Sample Data Time 0.00425	Per Sample DNN Time 1.03779	Train Loss 0.0689	
Epoch: [19][36/48]	Per Sample Total Time 1.04465	Per Sample Data Time 0.00428	Per Sample DNN Time 1.04037	Train Loss 0.0701	
Epoch: [19][46/48]	Per Sample Total Time 1.04370	Per Sample Data Time 0.00429	Per Sample DNN Time 1.03941	Train Loss 0.0830	
start validation
mAP: 0.752318
AUC: 0.600000
Avg Precision: 0.502165
Avg Recall: 1.000000
d_prime: 0.358287
train_loss: 0.089123
valid_loss: 0.809305
validation finished
Epoch-19 lr: 8.735421910125168e-07
epoch 19 training time: 402.975
---------------
2026-02-23 12:52:12.594380
current #epochs=20, #steps=912


z:\AST-With-TB-Classify\venvast\lib\site-packages\torch\cuda\amp\autocast_mode.py:118: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


Epoch: [20][8/48]	Per Sample Total Time 1.04483	Per Sample Data Time 0.00438	Per Sample DNN Time 1.04046	Train Loss 0.1418	
Epoch: [20][18/48]	Per Sample Total Time 1.04349	Per Sample Data Time 0.00437	Per Sample DNN Time 1.03912	Train Loss 0.0914	
Epoch: [20][28/48]	Per Sample Total Time 1.04190	Per Sample Data Time 0.00432	Per Sample DNN Time 1.03758	Train Loss 0.0714	
Epoch: [20][38/48]	Per Sample Total Time 1.04118	Per Sample Data Time 0.00432	Per Sample DNN Time 1.03686	Train Loss 0.0738	
start validation
mAP: 0.749540
AUC: 0.587500
Avg Precision: 0.502165
Avg Recall: 1.000000
d_prime: 0.312709
train_loss: 0.074390
valid_loss: 0.881169
validation finished
Epoch-20 lr: 7.425108623606393e-07
epoch 20 training time: 402.115
---------------Training Finished---------------
weighted averaged model results
mAP: 0.749599
AUC: 0.587500
Avg Precision: 0.504545
Avg Recall: 1.000000
d_prime: 0.312709
train_loss: 0.000000
valid_loss: 0.881169

  [✓] FOLD 5 training complete.
      Results  → z

In [4]:
# =============================================================================
# STEP 3: INFERENCE & EVALUATION — ทดสอบแต่ละ Fold บน Holdout Test Set
# =============================================================================
# กระบวนการต่อ 1 Fold:
#   1. โหลด Best Model (epoch ที่ mAUC สูงสุดบน VAL)
#   2. หา Optimal Threshold ด้วย Youden's J Index จาก VAL set
#   3. Run Inference บน Holdout Test Set (20% ที่ดึงออกไว้แต่แรก)
#   4. คำนวณ AUROC, AUPRC, Accuracy, Sensitivity, Specificity
#   5. แสดง AUC ของ Fold นั้น และสะสมเพื่อคำนวณ Mean AUC ใน Step 4
# =============================================================================

import torch
import torch.nn.functional as F
import numpy as np
import json
from torch.utils.data import DataLoader
from sklearn.metrics import (roc_auc_score, average_precision_score,
                             accuracy_score, confusion_matrix, roc_curve)

# Holdout test JSON (เดียวกันสำหรับทุก Fold)
HOLDOUT_JSON = os.path.join(FOLD_ROOT, "holdout_test.json")

all_fold_metrics = []   # สะสมผลทุก Fold → ใช้ใน Step 4

print("=" * 65)
print("  AST-P  |  5-FOLD INFERENCE ON SHARED HOLDOUT TEST SET")
print(f"  Holdout Test: {HOLDOUT_JSON}")
print("=" * 65)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----- โหลด Holdout Dataset (คงที่สำหรับทุก Fold) --------------------------
holdout_dataset = ast_dataloader.AudiosetDataset(
    HOLDOUT_JSON, audio_conf=EVAL_AUDIO_CONF, label_csv=LABEL_CSV)
holdout_loader  = DataLoader(
    holdout_dataset, batch_size=16, shuffle=False,
    num_workers=0, pin_memory=torch.cuda.is_available())

print(f"  Holdout Test: {len(holdout_dataset)} files  "
      f"[{sum(1 for _ in holdout_dataset if True)} total]")

# สรุป class distribution ใน holdout
with open(HOLDOUT_JSON) as f:
    hdata = json.load(f)["data"]
n_holdout_ptb    = sum(1 for r in hdata if r["labels"] == "1")
n_holdout_nonptb = len(hdata) - n_holdout_ptb
print(f"  Class distribution: {n_holdout_ptb} PTB / {n_holdout_nonptb} Non-PTB\n")

# ============================================================================
# INFERENCE LOOP
# ============================================================================
auroc_per_fold = []   # เก็บ AUROC ของแต่ละ Fold

for fold in range(1, N_FOLDS + 1):

    print(f"\n{'─'*65}")
    print(f"  FOLD {fold}  — Best Model by mAUC on VAL")
    print(f"{'─'*65}")

    fold_data_dir = os.path.join(FOLD_ROOT, f"fold_{fold}")
    fold_exp_dir  = os.path.join(EXP_ROOT,  f"fold_{fold}")

    # ------- 3.1  เลือก Best Epoch จาก result.csv (คอลัมน์ mAUC = index 1) ----
    result_csv_path = os.path.join(fold_exp_dir, "result.csv")
    result_matrix   = np.loadtxt(result_csv_path, delimiter=",")

    if result_matrix.ndim == 1:
        result_matrix = result_matrix.reshape(1, -1)

    mauc_per_epoch = result_matrix[:, 1]
    valid_mask     = mauc_per_epoch > 0
    best_epoch     = int(np.argmax(mauc_per_epoch * valid_mask)) + 1
    best_val_mauc  = float(mauc_per_epoch[best_epoch - 1])
    print(f"  Best epoch (VAL mAUC): Epoch {best_epoch:3d}  →  mAUC = {best_val_mauc:.4f}")

    # ------- 3.2  โหลด Model Checkpoint -------------------------------------------
    audio_model = models.ASTModel(
        label_dim=2, fstride=10, tstride=10,
        input_fdim=128, input_tdim=100,
        imagenet_pretrain=True, audioset_pretrain=True,
        model_size="base384", verbose=False
    )
    model_path = os.path.join(fold_exp_dir, "models", f"audio_model.{best_epoch}.pth")
    state_dict  = torch.load(model_path, map_location=device)
    if list(state_dict.keys())[0].startswith("module."):
        state_dict = {k[len("module."):]: v for k, v in state_dict.items()}
    audio_model.load_state_dict(state_dict)
    audio_model = audio_model.to(device).eval()
    print(f"  Model loaded: {model_path}")

    # ------- 3.3  Youden's J Threshold จาก VAL set --------------------------------
    val_json_path = os.path.join(fold_data_dir, "val_data.json")
    val_ds_inf    = ast_dataloader.AudiosetDataset(
        val_json_path, audio_conf=EVAL_AUDIO_CONF, label_csv=LABEL_CSV)
    val_ld_inf    = DataLoader(
        val_ds_inf, batch_size=16, shuffle=False,
        num_workers=0, pin_memory=torch.cuda.is_available())

    val_logits_list, val_labels_list = [], []
    with torch.no_grad():
        for aud_v, lbl_v in val_ld_inf:
            val_logits_list.append(audio_model(aud_v.to(device)).cpu())
            val_labels_list.append(lbl_v.cpu())

    val_probs_all = F.softmax(torch.cat(val_logits_list, dim=0), dim=1).numpy()
    val_prob_ptb  = val_probs_all[:, 1]
    val_targets   = torch.cat(val_labels_list, dim=0)
    y_val = (torch.argmax(val_targets, dim=1).numpy()
             if val_targets.dim() == 2 else val_targets.numpy().astype(int))

    try:
        fpr_v, tpr_v, thr_v = roc_curve(y_val, val_prob_ptb, pos_label=1)
        youden_j   = tpr_v - fpr_v
        best_j_idx = np.argmax(youden_j)
        opt_thresh = float(thr_v[best_j_idx])
        print(f"  Youden's J Threshold (VAL): {opt_thresh:.4f}  (J={youden_j[best_j_idx]:.4f})")
    except ValueError:
        opt_thresh = 0.5
        print(f"  Youden's J fallback (VAL 1-class): threshold = {opt_thresh:.4f}")

    # ------- 3.4  Inference บน Holdout Test Set ------------------------------------
    all_logits, all_targets_list = [], []
    with torch.no_grad():
        for audio_input, labels in holdout_loader:
            audio_input = audio_input.to(device)
            logits = audio_model(audio_input)
            all_logits.append(logits.cpu())
            all_targets_list.append(labels.cpu())

    all_logits  = torch.cat(all_logits,       dim=0)   # (N, 2)
    all_targets = torch.cat(all_targets_list, dim=0)   # (N, 2) one-hot

    probs_all = F.softmax(all_logits, dim=1).numpy()
    prob_ptb  = probs_all[:, 1]
    y_true    = (torch.argmax(all_targets, dim=1).numpy()
                 if all_targets.dim() == 2
                 else all_targets.numpy().astype(int))

    # ------- 3.5  Compute Metrics --------------------------------------------------
    auroc = roc_auc_score(y_true, prob_ptb)
    auprc = average_precision_score(y_true, prob_ptb)
    y_pred = (prob_ptb >= opt_thresh).astype(int)
    acc    = accuracy_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    auroc_per_fold.append(auroc)

    fold_result = {
        "fold":        fold,
        "auroc":       auroc,
        "auprc":       auprc,
        "accuracy":    acc,
        "sensitivity": sensitivity,
        "specificity": specificity,
        "threshold":   opt_thresh,
        "best_epoch":  best_epoch,
        "val_mauc":    best_val_mauc,
        "TP": int(tp), "TN": int(tn), "FP": int(fp), "FN": int(fn),
        "n_test":        int(len(y_true)),
        "n_ptb_test":    int(y_true.sum()),
        "n_nonptb_test": int((1 - y_true).sum()),
        "y_true":  y_true.tolist(),
        "y_pred":  y_pred.tolist(),
        "prob_ptb": prob_ptb.tolist(),
    }
    all_fold_metrics.append(fold_result)

    # บันทึกผล
    out_json_path = os.path.join(fold_exp_dir, "test_metrics.json")
    save_dict = {k: v for k, v in fold_result.items()
                 if k not in ("y_true", "y_pred", "prob_ptb")}
    with open(out_json_path, "w") as f:
        json.dump(save_dict, f, indent=4)

    # ------- 3.6  แสดงผล -----------------------------------------------------------
    print(f"\n  ┌──────────────────────────────────────────────────┐")
    print(f"  │  FOLD {fold}  RESULT  (Holdout Test Set)              │")
    print(f"  ├──────────────────────────────────────────────────┤")
    print(f"  │  AUROC        : {auroc:.4f}                            │")
    print(f"  │  AUPRC        : {auprc:.4f}                            │")
    print(f"  │  Accuracy     : {acc*100:.2f}%                           │")
    print(f"  │  Sensitivity  : {sensitivity*100:.2f}%  (TPR/Recall)        │")
    print(f"  │  Specificity  : {specificity*100:.2f}%  (TNR)               │")
    print(f"  │  Threshold*   : {opt_thresh:.4f}  (Youden's J on VAL)   │")
    print(f"  │  TP={tp:3d}  TN={tn:3d}  FP={fp:3d}  FN={fn:3d}              │")
    print(f"  └──────────────────────────────────────────────────┘")

# --------------------------------------------------------------------------- #
# 3.7  Summary: AUC ของแต่ละ Fold + Mean AUC
# --------------------------------------------------------------------------- #
auroc_arr  = np.array(auroc_per_fold)
mean_auroc = auroc_arr.mean()
std_auroc  = auroc_arr.std()

print(f"\n{'='*65}")
print("  AUC SUMMARY ACROSS FOLDS  (Holdout Test Set)")
print(f"{'='*65}")
print(f"  {'Fold':<8}  AUROC")
print(f"  {'─'*8}  {'─'*7}")
for i, auc in enumerate(auroc_per_fold, 1):
    bar = "█" * int(auc * 40)
    print(f"  Fold {i:<3}   {auc:.4f}  {bar}")
print(f"  {'─'*8}  {'─'*7}")
print(f"  {'Mean':<8}  {mean_auroc:.4f}  ± {std_auroc:.4f}")
print(f"{'='*65}")
print(f"\n[✓] All {N_FOLDS} Folds inference complete.")


  AST-P  |  5-FOLD INFERENCE ON SHARED HOLDOUT TEST SET
  Holdout Test: z:\AST-With-TB-Classify\json_folds\holdout_test.json
---------------the evaluation dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process audioset
use dataset mean -4.527 and std 5.118 to normalize the input.
number of classes is 2
  Holdout Test: 112 files  [112 total]
  Class distribution: 31 PTB / 81 Non-PTB


─────────────────────────────────────────────────────────────────
  FOLD 1  — Best Model by mAUC on VAL
─────────────────────────────────────────────────────────────────
  Best epoch (VAL mAUC): Epoch   3  →  mAUC = 0.4334
  Model loaded: z:\AST-With-TB-Classify\exp\kfold\fold_1\models\audio_model.3.pth
---------------the evaluation dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process audioset
use dataset mean -4.527 and std 5.118 to normalize the input.
number of classes is 2
  Youden'

In [5]:
# =============================================================================
# STEP 4: AGGREGATE RESULTS — Conclusion Evaluation + Confusion Matrix
# =============================================================================
# แสดง:
#   (A) Per-Fold AUROC Bar Chart
#   (B) Conclusion Evaluation Table: Mean ± Std ของ AUROC, AUPRC, Sens, Spec, Acc
#   (C) Macro Confusion Matrix (pooled across all folds) — Heatmap
#   (D) Clinical Interpretation
# =============================================================================

import numpy as np
import json
import os
import matplotlib
matplotlib.use("Agg")          # ใช้แบบ non-interactive; เปลี่ยนเป็น "TkAgg" ถ้าต้องการ popup
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec

# --------------------------------------------------------------------------- #
# 4.0  ตรวจสอบข้อมูล
# --------------------------------------------------------------------------- #
assert len(all_fold_metrics) == N_FOLDS, \
    f"Expected {N_FOLDS} fold results, got {len(all_fold_metrics)}. Run Step 3 first."

# --------------------------------------------------------------------------- #
# 4.1  แยก Metric Arrays
# --------------------------------------------------------------------------- #
aurocs        = np.array([m["auroc"]       for m in all_fold_metrics])
auprcs        = np.array([m["auprc"]       for m in all_fold_metrics])
accuracies    = np.array([m["accuracy"]    for m in all_fold_metrics])
sensitivities = np.array([m["sensitivity"] for m in all_fold_metrics])
specificities = np.array([m["specificity"] for m in all_fold_metrics])
thresholds    = np.array([m["threshold"]   for m in all_fold_metrics])
best_epochs   = np.array([m["best_epoch"]  for m in all_fold_metrics])
val_maucs     = np.array([m["val_mauc"]    for m in all_fold_metrics])

# Macro Confusion Matrix (pooled)
macro_TP = sum(m["TP"] for m in all_fold_metrics)
macro_TN = sum(m["TN"] for m in all_fold_metrics)
macro_FP = sum(m["FP"] for m in all_fold_metrics)
macro_FN = sum(m["FN"] for m in all_fold_metrics)
total_N  = macro_TP + macro_TN + macro_FP + macro_FN

macro_acc  = (macro_TP + macro_TN) / total_N if total_N > 0 else 0
macro_sens = macro_TP / (macro_TP + macro_FN) if (macro_TP + macro_FN) > 0 else 0
macro_spec = macro_TN / (macro_TN + macro_FP) if (macro_TN + macro_FP) > 0 else 0

# --------------------------------------------------------------------------- #
# 4.2  บันทึก Aggregate JSON
# --------------------------------------------------------------------------- #
aggregate = {
    "model":       "AST-P",
    "task":        "TB Screening (PTB vs Non-PTB)",
    "cv_strategy": f"{N_FOLDS}-Fold Patient-Level CV on 80% Pool + Shared Holdout Test (20%)",
    "n_folds":     N_FOLDS,
    "holdout_test_files": all_fold_metrics[0]["n_test"],
    "per_fold":    [{k: v for k, v in m.items()
                     if k not in ("y_true", "y_pred", "prob_ptb")}
                    for m in all_fold_metrics],
    "conclusion": {
        "AUROC":       {"mean": float(aurocs.mean()),       "std": float(aurocs.std())},
        "AUPRC":       {"mean": float(auprcs.mean()),       "std": float(auprcs.std())},
        "Accuracy":    {"mean": float(accuracies.mean()),   "std": float(accuracies.std())},
        "Sensitivity": {"mean": float(sensitivities.mean()),"std": float(sensitivities.std())},
        "Specificity": {"mean": float(specificities.mean()),"std": float(specificities.std())},
        "Threshold":   {"mean": float(thresholds.mean()),   "std": float(thresholds.std())},
    },
    "macro_confusion_matrix": {
        "TP": macro_TP, "TN": macro_TN, "FP": macro_FP, "FN": macro_FN,
        "Accuracy":    float(macro_acc),
        "Sensitivity": float(macro_sens),
        "Specificity": float(macro_spec),
    }
}
agg_output_path = os.path.join(BASE_DIR, "exp", "kfold", "aggregate_results.json")
os.makedirs(os.path.dirname(agg_output_path), exist_ok=True)
with open(agg_output_path, "w") as f:
    json.dump(aggregate, f, indent=4)

# --------------------------------------------------------------------------- #
# 4.3  Console Output
# --------------------------------------------------------------------------- #
SEP_THICK = "═" * 65
SEP_THIN  = "─" * 65

print(SEP_THICK)
print("  AST-P MODEL — 5-FOLD CV FINAL RESULTS")
print("  Task: Tuberculosis Screening  (PTB vs Non-PTB)")
print("  Test: Shared Holdout Set (20% of all patients)")
print(SEP_THICK)

# Per-Fold Table
print(f"\n  {'Fold':<6} {'AUROC':>7} {'AUPRC':>7} "
      f"{'Sens%':>7} {'Spec%':>7} {'Acc%':>7} {'Thresh':>8} {'BestEp':>7}")
print(f"  {'─'*6} {'─'*7} {'─'*7} {'─'*7} {'─'*7} {'─'*7} {'─'*8} {'─'*7}")
for m in all_fold_metrics:
    print(f"  {m['fold']:<6} "
          f"{m['auroc']:>7.4f} "
          f"{m['auprc']:>7.4f} "
          f"{m['sensitivity']*100:>7.2f} "
          f"{m['specificity']*100:>7.2f} "
          f"{m['accuracy']*100:>7.2f} "
          f"{m['threshold']:>8.4f} "
          f"{m['best_epoch']:>7d}")
print(f"  {'─'*6} {'─'*7} {'─'*7} {'─'*7} {'─'*7} {'─'*7} {'─'*8} {'─'*7}")
print(f"  {'Mean':<6} {aurocs.mean():>7.4f} {auprcs.mean():>7.4f} "
      f"{sensitivities.mean()*100:>7.2f} {specificities.mean()*100:>7.2f} "
      f"{accuracies.mean()*100:>7.2f} {thresholds.mean():>8.4f}")
print(f"  {'Std':<6} {aurocs.std():>7.4f} {auprcs.std():>7.4f} "
      f"{sensitivities.std()*100:>7.2f} {specificities.std()*100:>7.2f} "
      f"{accuracies.std()*100:>7.2f} {thresholds.std():>8.4f}")

# Conclusion Summary
print(f"\n{SEP_THICK}")
print("  CONCLUSION EVALUATION  (Mean ± Std,  N=5 Folds)")
print(SEP_THICK)
metrics_report = [
    ("AUROC",        aurocs,        "%"),
    ("AUPRC",        auprcs,        "%"),
    ("Accuracy",     accuracies,    "%"),
    ("Sensitivity",  sensitivities, "%"),
    ("Specificity",  specificities, "%"),
    ("Threshold",    thresholds,    ""),
]
for name, arr, unit in metrics_report:
    scale = 100 if unit == "%" else 1
    print(f"  {name:<16} : {arr.mean()*scale:.2f}{unit}  ±  {arr.std()*scale:.2f}{unit}")

# Macro Confusion Matrix — Text
print(f"\n{SEP_THICK}")
print("  MACRO CONFUSION MATRIX  (Pooled across all Folds × Holdout Test)")
print(SEP_THICK)
print(f"  {'':22}  Predicted")
print(f"  {'Actual':22}  Non-PTB   PTB")
print(f"  {'─'*40}")
print(f"  {'Non-PTB (Label=0)':22}  TN={macro_TN:4d}   FP={macro_FP:4d}")
print(f"  {'PTB     (Label=1)':22}  FN={macro_FN:4d}   TP={macro_TP:4d}")
print(f"  {'─'*40}")
print(f"  Macro Accuracy    : {macro_acc*100:.2f}%")
print(f"  Macro Sensitivity : {macro_sens*100:.2f}%")
print(f"  Macro Specificity : {macro_spec*100:.2f}%")

# Clinical Interpretation
print(f"\n{SEP_THICK}")
print("  CLINICAL INTERPRETATION")
print(SEP_THIN)
sens_mean = sensitivities.mean()
spec_mean = specificities.mean()
sens_rate = ("Excellent (≥85%) — suitable for screening" if sens_mean >= 0.85
             else "Good (≥75%) — acceptable for assisted screening" if sens_mean >= 0.75
             else "Needs improvement (<75%)")
spec_rate = ("Excellent (≥80%) — low false alarm rate" if spec_mean >= 0.80
             else "Acceptable (≥65%)" if spec_mean >= 0.65
             else "Low (<65%) — high false alarm rate")
print(f"  Sensitivity  → {sens_rate}")
print(f"  Specificity  → {spec_rate}")
print(f"\n  Youden's J threshold (from VAL) was applied per fold.")
print(f"  All folds were evaluated on the same Holdout Test Set (20%).")
print(SEP_THICK)

# --------------------------------------------------------------------------- #
# 4.4  Visualizations
# --------------------------------------------------------------------------- #
folds      = [m["fold"] for m in all_fold_metrics]
colors_bar = ["#4C72B0" if a >= aurocs.mean() else "#DD8452" for a in aurocs]

fig = plt.figure(figsize=(16, 12))
fig.suptitle("AST-P | 5-Fold CV Results (Holdout Strategy)",
             fontsize=15, fontweight="bold", y=0.98)
gs = GridSpec(2, 2, figure=fig, hspace=0.45, wspace=0.38)

# ── Plot A: Per-Fold AUROC Bar Chart ─────────────────────────────────────────
ax_auc = fig.add_subplot(gs[0, 0])
bars = ax_auc.bar(folds, aurocs, color=colors_bar, edgecolor="white", linewidth=0.8)
ax_auc.axhline(aurocs.mean(), color="crimson", linewidth=1.5,
               linestyle="--", label=f"Mean AUROC = {aurocs.mean():.4f}")
ax_auc.fill_between([0.4, N_FOLDS + 0.6],
                    aurocs.mean() - aurocs.std(),
                    aurocs.mean() + aurocs.std(),
                    color="crimson", alpha=0.10, label=f"± 1 SD")
for bar, val in zip(bars, aurocs):
    ax_auc.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.005,
                f"{val:.4f}", ha="center", va="bottom", fontsize=9)
ax_auc.set_title("AUROC per Fold", fontweight="bold")
ax_auc.set_xlabel("Fold")
ax_auc.set_ylabel("AUROC")
ax_auc.set_ylim(max(0, aurocs.min() - 0.1), min(1.05, aurocs.max() + 0.12))
ax_auc.set_xticks(folds)
ax_auc.legend(fontsize=8)
ax_auc.grid(axis="y", linestyle="--", alpha=0.4)

# ── Plot B: Conclusion Metrics (Mean ± Std) Bar Chart ────────────────────────
ax_met = fig.add_subplot(gs[0, 1])
metric_names = ["AUROC", "AUPRC", "Accuracy", "Sensitivity", "Specificity"]
means = [aurocs.mean(), auprcs.mean(), accuracies.mean(),
         sensitivities.mean(), specificities.mean()]
stds  = [aurocs.std(),  auprcs.std(),  accuracies.std(),
         sensitivities.std(),  specificities.std()]
x_pos = np.arange(len(metric_names))
color_met = ["#4C72B0", "#55A868", "#C44E52", "#8172B2", "#937860"]
err_kw    = dict(ecolor="black", capsize=5, linewidth=1.2)
ax_met.bar(x_pos, means, yerr=stds, color=color_met,
           edgecolor="white", linewidth=0.8, error_kw=err_kw)
for i, (m_val, s_val) in enumerate(zip(means, stds)):
    ax_met.text(i, m_val + s_val + 0.015,
                f"{m_val*100:.1f}%\n±{s_val*100:.1f}%",
                ha="center", va="bottom", fontsize=7.5)
ax_met.set_title("Conclusion: Mean ± Std", fontweight="bold")
ax_met.set_xticks(x_pos)
ax_met.set_xticklabels(metric_names, fontsize=9)
ax_met.set_ylabel("Score")
ax_met.set_ylim(0, 1.25)
ax_met.axhline(0.5, color="gray", linewidth=0.8, linestyle=":")
ax_met.grid(axis="y", linestyle="--", alpha=0.4)

# ── Plot C: Macro Confusion Matrix Heatmap ────────────────────────────────────
ax_cm = fig.add_subplot(gs[1, :])
cm_matrix = np.array([[macro_TN, macro_FP],
                       [macro_FN, macro_TP]])
im = ax_cm.imshow(cm_matrix, interpolation="nearest", cmap="Blues")
plt.colorbar(im, ax=ax_cm, fraction=0.046, pad=0.04)
tick_labels = ["Non-PTB (0)", "PTB (1)"]
ax_cm.set_xticks([0, 1]);  ax_cm.set_xticklabels(tick_labels, fontsize=11)
ax_cm.set_yticks([0, 1]);  ax_cm.set_yticklabels(tick_labels, fontsize=11)
ax_cm.set_xlabel("Predicted Label", fontsize=12)
ax_cm.set_ylabel("True Label", fontsize=12)
ax_cm.set_title(
    f"Macro Confusion Matrix  (Pooled, {N_FOLDS} Folds × Holdout Test)\n"
    f"Accuracy={macro_acc*100:.1f}%  |  Sensitivity={macro_sens*100:.1f}%  |  Specificity={macro_spec*100:.1f}%",
    fontweight="bold", fontsize=11)

cell_text = [[f"TN\n{macro_TN}", f"FP\n{macro_FP}"],
             [f"FN\n{macro_FN}", f"TP\n{macro_TP}"]]
cm_norm   = cm_matrix / (cm_matrix.max() + 1e-9)
for i in range(2):
    for j in range(2):
        text_color = "white" if cm_norm[i, j] > 0.5 else "black"
        ax_cm.text(j, i, cell_text[i][j],
                   ha="center", va="center",
                   fontsize=13, fontweight="bold", color=text_color)

# Custom legend patches
tp_patch = mpatches.Patch(color="#2171b5",  label="TP: PTB correctly identified")
tn_patch = mpatches.Patch(color="#c6dbef",  label="TN: Non-PTB correctly identified")
fp_patch = mpatches.Patch(color="#fdae6b",  label="FP: Non-PTB predicted as PTB")
fn_patch = mpatches.Patch(color="#e6550d",  label="FN: PTB missed (predicted Non-PTB)")
ax_cm.legend(handles=[tp_patch, tn_patch, fp_patch, fn_patch],
             loc="lower right", fontsize=8, framealpha=0.85)

# Save figure
plot_dir = os.path.join(BASE_DIR, "exp", "kfold", "plots")
os.makedirs(plot_dir, exist_ok=True)
plot_path = os.path.join(plot_dir, "aggregate_results.png")
plt.savefig(plot_path, dpi=150, bbox_inches="tight", facecolor="white")
plt.show()
print(f"\n[✓] Plot saved → {plot_path}")
print(f"[✓] Results JSON → {agg_output_path}")
print(f"[✓] Pipeline complete.")


═════════════════════════════════════════════════════════════════
  AST-P MODEL — 5-FOLD CV FINAL RESULTS
  Task: Tuberculosis Screening  (PTB vs Non-PTB)
  Test: Shared Holdout Set (20% of all patients)
═════════════════════════════════════════════════════════════════

  Fold     AUROC   AUPRC   Sens%   Spec%    Acc%   Thresh  BestEp
  ────── ─────── ─────── ─────── ─────── ─────── ──────── ───────
  1       0.6002  0.3176   96.77   11.11   34.82   0.2855       3
  2       0.6623  0.4140   87.10   46.91   58.04   0.1701       4
  3       0.6396  0.4099   77.42   50.62   58.04   0.0594      14
  4       0.6933  0.5326   35.48   91.36   75.89   0.8246       2
  5       0.6460  0.4251   74.19   49.38   56.25   0.5430       1
  ────── ─────── ─────── ─────── ─────── ─────── ──────── ───────
  Mean    0.6483  0.4198   74.19   49.88   56.61   0.3765
  Std     0.0304  0.0683   20.91   25.43   13.04   0.2757

═════════════════════════════════════════════════════════════════
  CONCLUSION EVALU

C:\Users\MxnlodyX\AppData\Local\Temp\ipykernel_33772\215560960.py:256: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
